# Creds

> Fill in a module description here


In [100]:
# | default_exp google.auth

In [101]:
# | exporti

import os
import json
from dataclasses import dataclass, field
from typing import List

from dotenv import load_dotenv, set_key

from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google.auth.exceptions import RefreshError

from nbdev.showdoc import patch_to

In [102]:
# | export

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import Resource

In [103]:
# | exporti

default_scope = [
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive.metadata.readonly",
    "https://www.googleapis.com/auth/drive.readonly",
]

In [104]:
# | export


@dataclass
class GoogleAuth:
    creds: Credentials = None
    service: Resource = None

    scope: List[str] = field(default_factory=lambda: default_scope)

    def _refresh_or_new_token(self, flow) -> Credentials:
        """Refreshes the token or creates a new one if it does not exist"""

        if self.creds and isinstance(self.creds, Credentials) and self.creds.valid:
            pass

        elif (
            self.creds
            and isinstance(self.creds, Credentials)
            and self.creds.expired
            and self.creds.refresh_token
        ):
            # if creds token can be refreshed, do it.
            try:
                print("refreshing creds using saved token")
                self.creds.refresh(Request())

            except RefreshError as e:
                print(e)
                self.creds.refresh(Request())

        else:
            # If there are no (valid) credentials available, run the Oauth flow
            print("generating a brand new token cred via auth flow")

            self.creds = flow.run_local_server(port=0)

        self.generate_service(service_name="drive", service_version="v3")

        return self.creds

    def generate_service(self, service_name="drive", service_version="v3") -> Resource:
        """Generates a service object for the given service name and version"""

        print(f"generating service object on {self.__class__.__name__}")

        self.service = build(
            serviceName=service_name, version=service_version, credentials=self.creds
        )

        return self.service

In [105]:
# |exporti


@patch_to(GoogleAuth)
def get_creds_from_file(
    self: GoogleAuth,
    token_file_path="env_token.json",
    credentials_file_path="env_creds.json",
    scope=None,
) -> Credentials:
    """
    generate creds here, https://console.cloud.google.com/apis/credentials/oauthclient
    for a new implementation you'll need to
    1. set up a project at console.cloud.google.com,
    2. enable (turn on) the appropriate APIs for that project
    3. configure OAuth access (your python script will use an Oauth token to authenticate, so your project needs the OAuth screen)
    4. download the creds file and store it as env_creds.json
    """

    scope = scope or self.scope

    if token_file_path and os.path.exists(token_file_path):
        print(f"using saved token {token_file_path}")
        self.creds = Credentials.from_authorized_user_file(token_file_path, scope)

    self.flow = InstalledAppFlow.from_client_secrets_file(credentials_file_path, scope)

    return self._refresh_or_new_token(self.flow)

In [106]:
# get_creds_from file requires the existence of a locally saved credentials file.
# The auth flow will produce a token file (which facilitates refresh) if a token file doesn't exist

load_dotenv(".env", override=True)

token_file_path = "env_token.json"
credentials_file_path = "env_creds.json"

if not os.path.exists(credentials_file_path) and os.environ.get("GDOC_KEY"):
    with open(credentials_file_path, "w") as f:
        json.dump(json.loads(os.environ["GDOC_KEY"]), f)

if not os.path.exists(token_file_path) and os.environ.get("GDOC_TOKEN"):
    with open(token_file_path, "w") as f:
        json.dump(json.loads(os.environ["GDOC_TOKEN"]), f)

if os.path.exists(token_file_path) or os.path.exists(credentials_file_path):
    google_auth = GoogleAuth()
    creds = google_auth.get_creds_from_file(
        credentials_file_path=credentials_file_path, token_file_path=token_file_path
    )
    print(creds)

using saved token env_token.json
refreshing creds using saved token
generating service object on GoogleAuth


In [107]:
# | exporti


def _load_creds_from_env(token_env_key, creds_env_key):
    token_str = os.getenv(token_env_key)
    creds_str = os.getenv(creds_env_key)

    token_json = (
        json.loads(token_str) if token_str and isinstance(token_str, str) else token_str
    )
    creds_json = (
        json.loads(creds_str) if creds_str and isinstance(creds_str, str) else creds_str
    )

    return token_json, creds_json

In [108]:
google_auth = GoogleAuth()
token_json, creds_json = _load_creds_from_env(
    token_env_key="GDOC_TOKEN", creds_env_key="GDOC_KEY"
)
assert token_json, creds_json

In [109]:
# | exporti


def _update_env(env_file=".env", key=None, value=None):
    """
    updates a .env file with a key value pair
    then reloads the env_file
    """

    if not os.path.exists(env_file):
        with open(env_file, "w", encoding="utf-8") as f:
            f.write("")

    if key and value:
        set_key(env_file, key, value)

    load_dotenv(env_file, override=True)

In [110]:
import datetime as dt

google_auth = GoogleAuth()

env_file = ".env"
prop_name = "test_update_env"

_update_env(env_file=env_file, key=prop_name, value=f"updated - {dt.date.today()}")

load_dotenv(".env", override=True)

{prop_name: os.getenv(prop_name)}

{'test_update_env': 'updated - 2023-12-02'}

In [111]:
# |exporti


@patch_to(GoogleAuth)
def get_creds_from_env(
    self: GoogleAuth,
    scope=None,
    token_env_key: str = "GDOC_TOKEN",
    credentials_env_key: str = "GDOC_CLIENT",
    env_file=".env",
):
    """
    credentials should be stored in the .env file as a json (NOT A STRING)
    token will be stored as a string (NOT A JSON OBJ)

    generate creds here, https://console.cloud.google.com/apis/credentials/oauthclient
    for a new implementation you'll need to
    1. set up a project at console.cloud.google.com,
    2. enable (turn on) the appropriate APIs for that project
    3. configure OAuth access (your python script will use an Oauth token to authenticate, so your project needs the OAuth screen)
    4. download the creds file and store it as env_creds.json
    """

    token_json, credentials_json = _load_creds_from_env(
        token_env_key, credentials_env_key
    )

    if token_json:
        print("using saved token")
        self.creds = Credentials.from_authorized_user_info(token_json, scope)

    self.flow = InstalledAppFlow.from_client_config(credentials_json, self.scope)

    self._refresh_or_new_token(self.flow)

    # Save the credentials for the next run
    _update_env(env_file, token_env_key, self.creds.to_json())

    return self.creds

#### sample implementation of get_creds_from_env


In [112]:
google_auth = GoogleAuth()
google_auth.get_creds_from_env(
    credentials_env_key="GDOC_KEY", token_env_key="GDOC_TOKEN", env_file=".env"
)

using saved token
generating service object on GoogleAuth


In [113]:
# | hide
import nbdev

nbdev.nbdev_export()